In [22]:

import os
import dashscope
from serpapi import GoogleSearch
from dashscope import Generation


dashscope.api_key = 'sk-85910f047d284b8c8bb562dc6c9f41ce'  
SERPAPI_API_KEY = 'cdc4857354353a39bfb262095d4f283666a7ac68f855428fab5931d94e44a7ac'




In [29]:
def search_digestive_diseases(query, num_results=10):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "num": num_results,
        "gl": "us", 
        "hl": "en"  
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    snippets = []
    for result in results.get("organic_results", []):
        if 'snippet' in result:
            snippets.append(result['snippet'])
    return snippets

def extract_keywords(query):
    prompt = f"""
please extract keywords from the query，and split the keywords with comma：
query：{query}
keywords：
"""
    
    response = Generation.call(
        model='qwen-turbo',
        prompt=prompt,
        temperature=0.3,
        max_tokens=100,
        result_format='message'
    )
    
    if response.status_code == 200:
        keywords = response.output.choices[0].message.content.strip()
        return keywords
    else:
        return f"Error: {response.message}"
    
def filter_with_qwen(texts, question):

    combined_text = "\n".join(texts)
    prompt = f"""
You are an expert in digestive diseases. Please select the paragraphs most relevant to the "{question}" from the following content and remove the unrelated paragraphs.
Return only the selected content without any explanations or summaries.
content：
{combined_text}
filtered text：
"""
    
    response = Generation.call(
        model='qwen-turbo',
        prompt=prompt,
        temperature=0.3,
        max_tokens=1000,
        result_format='message'
    )
    
    if response.status_code == 200:
        filtered_text = response.output.choices[0].message.content
        return filtered_text
    else:
        return f"Error: {response.message}"

def summarize_with_qwen(text, question):
    prompt = f"""
please summarize the filtered content and use simple and easy-to-understand language for audiences with low literacy.
content：
{text}
summary：
"""
    response = Generation.call(
        model='qwen-turbo',
        prompt=prompt,
        temperature=0.3,
        max_tokens=500,
        result_format='message'
    )
    
    if response.status_code == 200:
        summary = response.output.choices[0].message.content
        return summary
    else:
        return f"Error: {response.message}"

def main():
    query = "If I had diarrhea, what could be the possible cause?"
    
    print("=== extrct keywords ===")
    keywords = extract_keywords(query)
    print(f"query：{query}")
    print(f"extracted keywords：{keywords}")
    
    print(f"start searching for keywords: {keywords}")
    search_results = search_digestive_diseases(keywords)
    print(f"get {len(search_results)} related results。")

    print("\n=== filter the results ===")
    filtered_text = filter_with_qwen(search_results, keywords)
    print("filtered text：")
    print(filtered_text)

    print("\n=== summarize results ===")
    summary = summarize_with_qwen(filtered_text, keywords)
    print("summary：")
    print(summary)
    
    
if __name__ == "__main__":
    main()

=== extrct keywords ===
query：If I had diarrhea, what could be the possible cause?
extracted keywords：diarrhea, possible cause
start searching for keywords: diarrhea, possible cause
get 8 related results。

=== filter the results ===
filtered text：
Causes · Viruses. Viruses that can cause diarrhea include Norwalk virus, also known as norovirus, enteric adenoviruses, astrovirus, ...
Common causes of diarrhea include medications, food intolerances and food poisoning. Germs, diseases affecting your gut and even treatments can all cause ...
What causes diarrhea? · A bacterial infection · A virus · Trouble digesting certain things (food intolerance) · Food allergy (such as celiac disease, gluten allergy) ...
Common causes of diarrhea include infections, food allergies, and food poisoning. It's often due to a virus infecting your gut, but it can also ...
The most common causes of acute diarrhea are viral gastroenteritis, food poisoning, and medicine side effects.
Viral and bacterial infection